In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

### Positional indexing

In [2]:
da = xr.DataArray(np.random.rand(4, 3),
                  [('time', pd.date_range('2000-01-01', periods=4)),
                   ('space', ['WA', 'OR', 'CA'])])
da[:2]

<xarray.DataArray (time: 2, space: 3)>
array([[0.24553368, 0.72290409, 0.53604832],
       [0.98699327, 0.73117117, 0.97346691]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02
  * space    (space) <U2 'WA' 'OR' 'CA'

In [3]:
da[0, 0]

<xarray.DataArray ()>
array(0.24553368)
Coordinates:
    time     datetime64[ns] 2000-01-01
    space    <U2 'WA'

In [4]:
da[:, [2, 1]]

<xarray.DataArray (time: 4, space: 2)>
array([[0.53604832, 0.72290409],
       [0.97346691, 0.73117117],
       [0.11921205, 0.6975578 ],
       [0.55813249, 0.72686274]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'CA' 'OR'

In [5]:
da.loc['2000-01-01':'2000-01-02', 'OR']

<xarray.DataArray (time: 2)>
array([0.72290409, 0.73117117])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02
    space    <U2 'OR'

In [6]:
da.loc['2000-01-01':'2000-01-02', ['WA', 'CA']] = 0
da

<xarray.DataArray (time: 4, space: 3)>
array([[0.        , 0.72290409, 0.        ],
       [0.        , 0.73117117, 0.        ],
       [0.83803743, 0.6975578 , 0.11921205],
       [0.91282381, 0.72686274, 0.55813249]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'WA' 'OR' 'CA'

### Indexing with dim names

In [7]:
da[dict(space=0, time=slice(None, 2))]

<xarray.DataArray (time: 2)>
array([0., 0.])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02
    space    <U2 'WA'

In [8]:
da.loc[{'time': slice('2000-01-01', '2000-01-01')}]

<xarray.DataArray (time: 1, space: 3)>
array([[0.        , 0.72290409, 0.        ]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01
  * space    (space) <U2 'WA' 'OR' 'CA'

In [9]:
da.isel(space=0, time=slice(None, 2))

<xarray.DataArray (time: 2)>
array([0., 0.])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02
    space    <U2 'WA'

In [10]:
da.sel(time=slice('2000-01-01', '2000-01-01'))

<xarray.DataArray (time: 1, space: 3)>
array([[0.        , 0.72290409, 0.        ]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01
  * space    (space) <U2 'WA' 'OR' 'CA'

### Nearest neighbor lookups

In [11]:
da = xr.DataArray([1, 2, 3], [('x', [0, 1, 2])])
da.sel(x=[1.1, 1.9], method='nearest')

<xarray.DataArray (x: 2)>
array([2, 3])
Coordinates:
  * x        (x) int64 1 2

In [12]:
da.sel(x=0.1, method='backfill')

<xarray.DataArray ()>
array(2)
Coordinates:
    x        int64 1

In [13]:
da.reindex(x=[1.1, 1.5], method='nearest', tolerance=0.2)

<xarray.DataArray (x: 2)>
array([ 2., nan])
Coordinates:
  * x        (x) float64 1.1 1.5

In [14]:
da.sel(x=slice(1, 3), method='nearest') # not (yet) supported

NotImplementedError: cannot use ``method`` argument if any indexers are slice objects

In [16]:
da.sel(x=slice(0.9, 3.1))

<xarray.DataArray (x: 2)>
array([2, 3])
Coordinates:
  * x        (x) int64 1 2

In [17]:
rev_da = da[::-1]
rev_da.loc[3.1:0.9]

<xarray.DataArray (x: 2)>
array([3, 2])
Coordinates:
  * x        (x) int64 2 1

### Dataset Indexing

In [19]:
da = xr.DataArray(np.random.rand(4, 3),
                  [('time', pd.date_range('2000-01-01', periods=4)),
                   ('space', ['WA', 'OR', 'CA'])])
ds = da.to_dataset(name='foo')
ds.isel(space=[1], time=[0])

<xarray.Dataset>
Dimensions:  (space: 1, time: 1)
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01
  * space    (space) <U2 'OR'
Data variables:
    foo      (time, space) float64 0.03609

In [20]:
ds.sel(time='2000-01-02')

<xarray.Dataset>
Dimensions:  (space: 3)
Coordinates:
    time     datetime64[ns] 2000-01-02
  * space    (space) <U2 'WA' 'OR' 'CA'
Data variables:
    foo      (space) float64 0.3806 0.7454 0.09318

In [23]:
ds[{'space': [0, 2], 'time': [2]}]

<xarray.Dataset>
Dimensions:  (space: 2, time: 1)
Coordinates:
  * time     (time) datetime64[ns] 2000-01-03
  * space    (space) <U2 'WA' 'CA'
Data variables:
    foo      (time, space) float64 0.9361 0.8479

In [25]:
ds.loc[{'time': '2000-01-01'}]

<xarray.Dataset>
Dimensions:  (space: 3)
Coordinates:
    time     datetime64[ns] 2000-01-01
  * space    (space) <U2 'WA' 'OR' 'CA'
Data variables:
    foo      (space) float64 0.3213 0.03609 0.7751

### Dropping labels/dimensions

In [26]:
ds.drop_sel(space=['WA', 'OR'])

<xarray.Dataset>
Dimensions:  (space: 1, time: 4)
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'CA'
Data variables:
    foo      (time, space) float64 0.7751 0.09318 0.8479 0.9668

In [27]:
ds.drop_dims('time')

<xarray.Dataset>
Dimensions:  (space: 3)
Coordinates:
  * space    (space) <U2 'WA' 'OR' 'CA'
Data variables:
    *empty*

### Masking with `where`

In [29]:
da = xr.DataArray(np.arange(16).reshape(4, 4), dims=['x', 'y'])
da.where(da.x + da.y <= 4)

<xarray.DataArray (x: 4, y: 4)>
array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., nan],
       [12., 13., nan, nan]])
Dimensions without coordinates: x, y

In [30]:
da.where(da.y < 2)

<xarray.DataArray (x: 4, y: 4)>
array([[ 0.,  1., nan, nan],
       [ 4.,  5., nan, nan],
       [ 8.,  9., nan, nan],
       [12., 13., nan, nan]])
Dimensions without coordinates: x, y

In [31]:
da.where(da.y < 2, drop=True)

<xarray.DataArray (x: 4, y: 2)>
array([[ 0.,  1.],
       [ 4.,  5.],
       [ 8.,  9.],
       [12., 13.]])
Dimensions without coordinates: x, y

### Selecting values with `isin`